In [1]:
from __future__ import print_function
from fenics import *
from ufl import nabla_div
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator
from mpl_toolkits.mplot3d import Axes3D
import petsc4py
petsc4py.init()
from petsc4py import PETSc
%matplotlib inline


# PETSc Tutorial 

https://computationalmechanics.in/vector-and-matrix-operations-in-petsc-via-petsc4py/

In [3]:
n = 10 # Size of vector
x = PETSc.Vec().createSeq(n) # Faster way to create a sequential vector.
x.setValues(range(n), range(n))
print(x.getArray())
print(x.getValues(3))
print(x.getValues([1, 2]))

[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9.]
3.0
[ 1.  2.]


In [4]:
#add scalar to a vector
x.setValues(range(n), range(n))
x.shift(1)
print(x.getArray())
x.shift(-1)
print(x.getArray())

[  1.   2.   3.   4.   5.   6.   7.   8.   9.  10.]
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9.]


In [6]:
#adding 2 vectors
x.setValues(range(n), range(n))
y = PETSc.Vec().createSeq(n)
y.setValues(range(n), range(n))
z=x+y
print(z.getArray())

[  0.   2.   4.   6.   8.  10.  12.  14.  16.  18.]


In [7]:
#sum entries of vector
print(x.sum())
print(x.min())
print(x.max())

45.0
(0, 0.0)
(9, 9.0)


In [8]:
#vector dot product
print(x.dot(x)) # dot product with self
print(x.dot(y)) # dot product with another vector

285.0
285.0


In [9]:
print ('2-norm =', x.norm())
print ('Infinity-norm =', x.norm(PETSc.NormType.NORM_INFINITY))

2-norm = 16.881943016134134
Infinity-norm = 9.0


In [10]:
m, n = 4, 4 # size of the matrix
A = PETSc.Mat().createAIJ([m, n]) # AIJ represents sparse matrix
A.setUp()
A.assemble()

In [11]:
#use getValues method to print
print(A.getValues(range(m), range(n)))
print(A.getValues(range(2), range(1)))

[[ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]]
[[ 0.]
 [ 0.]]


In [15]:
#setting values to matrix
#setValue(self, row, col, value, addv=None) will assign the value to the mentioned [row][col] index of the matrix 
#and setValues(self, rows, cols, values, addv=None) will assign values to the positions obtained from the tensor product of rows and cols.
A.setValue(1, 1, -9)
A.setValue(0, 0, -2)
A.setValue(2, 2, -5)
A.setValue(3, 3, 6)
#looks like row major
A.setValues([0, 1], [2, 3], [1, 2, 4, 8])
A.assemble()
print(A.getValues(range(m), range(n)))

[[-2.  0.  1.  2.]
 [ 0. -9.  4.  8.]
 [ 0.  0. -5.  0.]
 [ 0.  0.  0.  6.]]


In [16]:
#get size and transpose
print(A.getSize())
B = A.copy()
B.transpose()
print(A.getSize(), B.getSize())
print(B.getValues(range(4), range(4)))


(4, 4)
(4, 4) (4, 4)
[[-2.  0.  0.  0.]
 [ 0. -9.  0.  0.]
 [ 1.  4. -5.  0.]
 [ 2.  8.  0.  6.]]


In [17]:
#matrix multiplication
C = A.matMult(B)
print(C.getValues(range(m), range(n)))

[[   9.   20.   -5.   12.]
 [  20.  161.  -20.   48.]
 [  -5.  -20.   25.    0.]
 [  12.   48.    0.   36.]]


In [19]:
#mult(self, Vec x, Vec y) multiply matrix with vector x where y is the resulting vector
x = PETSc.Vec().createSeq(4) # making the x vector
x.set(1) # assigning value 1 to all the elements
y = PETSc.Vec().createSeq(4) # Put answer here.
A.mult(x, y) # A*x = y
print(y.getArray())

[ 1.  3. -5.  6.]


In [20]:
#most simple solver is krylov
print("Matrix A: ")
print(A.getValues(range(m), range(n))) # printing the matrix A defined above

b = PETSc.Vec().createSeq(4) # creating a vector
b.setValues(range(4), [10, 5, 3, 6]) # assigning values to the vector

print('\\n Vector b: ')
print(b.getArray()) # printing the vector 

x = PETSc.Vec().createSeq(4) # create the solution vector x

ksp = PETSc.KSP().create() # creating a KSP object named ksp
ksp.setOperators(A)

# Allow for solver choice to be set from command line with -ksp_type <solver>.
ksp.setFromOptions()
print ('\\n Solving with:', ksp.getType()) # prints the type of solver

# Solve!
ksp.solve(b, x) 

print('\\n Solution vector x: ')
print(x.getArray())

Matrix A: 
[[-2.  0.  1.  2.]
 [ 0. -9.  4.  8.]
 [ 0.  0. -5.  0.]
 [ 0.  0.  0.  6.]]
\n Vector b: 
[ 10.   5.   3.   6.]
\n Solving with: gmres
\n Solution vector x: 
[-4.3         0.06666667 -0.6         1.        ]


In [12]:
#now using PETSc interface to FEniCS
nx = 10
mesh = UnitIntervalMesh(nx)
V=FunctionSpace(mesh,'P',1)
u = TrialFunction(V)
v = TestFunction(V)

k_form = u*v*dx
f_form = v*dx

K = PETScMatrix()

assemble(k_form, tensor=K)
print(type(K))
#change to petsc objet
Kp = as_backend_type(K).mat()
print(type(Kp))


#how to assemble vector?
F = PETScVector()
assemble(f_form, tensor=F)
print(type(F))

Fp = as_backend_type(F).vec()
print(type(Fp))


<class 'dolfin.cpp.la.PETScMatrix'>
<class 'petsc4py.PETSc.Mat'>
<class 'dolfin.cpp.la.PETScVector'>
<class 'petsc4py.PETSc.Vec'>


In [6]:
#other case, if I have a numpy matrix X
X = np.eye(20)
Xp = PETSc.Mat().createAIJ(X.T.shape)
Xp.setUp()
Xp.setValues(range(0, X.shape[1]), range(0, X.shape[0]), X.T) # copy X matrix into petsc matrix
Xp.assemble()
Xp.transpose()
print(Xp.getValues(range(0, X.shape[0]), range(0, X.shape[1])))

[[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.]
 [ 0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.
   0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.

In [5]:
print(X.shape[1])

20
